In [1]:
# General libraries
# import jax
import sys


import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from IPython.display import clear_output

# Own code
sys.path.append("../")
from utils.data_utils import create_data, create_dgp_data, transformation, standardize
from utils.tvp_models import TVPVARModel

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

# Set RNG
np.random.seed(12345)

In [25]:
train = 178 # 2005Q1

M = 7
p = 4
T = train - p
k = M*(M*p+1)

print(f'k: {k}')

k: 203


In [26]:
ds = pd.read_csv("../data/fred_qd.csv")

In [27]:
transform = True

gdp = transformation(ds["GDPC1"].iloc[2:].to_numpy(), 5, transform, scale=100)
cpi = transformation(ds["CPIAUCSL"].iloc[2:].to_numpy(), 6, transform, scale=100)
fedfund = transformation(ds["FEDFUNDS"].iloc[2:].to_numpy(), 2, transform, scale=100)
compi = transformation(ds["PPIACO"].iloc[2:].to_numpy(), 6, transform, scale=100)
borrowings = transformation(ds["TOTRESNS"].iloc[2:].to_numpy(), 6, transform, scale=100)
sp500 = transformation(ds["S&P 500"].iloc[2:].to_numpy(), 5, transform, scale=100)
m2 = transformation(ds["M2REAL"].iloc[2:].to_numpy(), 5, transform, scale=100)

# Start due to transformation

lag = 2

# series_total = [gdp[lag:], cpi[lag:], fedfund[lag:]]
series_total = [gdp[lag:], cpi[lag:], fedfund[lag:], compi[lag:], borrowings[lag:], sp500[lag:], m2[lag:]]

In [28]:
standardized_series = standardize(series_total, train)

### TVP-VAR with Variational Bayes (VB) - M = 7

In [29]:
complete_T = len(gdp[lag:])
X_complete, y_complete = create_data(standardized_series,
                                     complete_T,
                                     complete_T-p,
                                     M,
                                     p,
                                     k)

In [7]:
# SVSS
# Optimizing the shrinkage for M = 7 (default values are optimal for M = 3)

# tau_0 = np.arange(0.5, 5, 1)
# tau_1 = np.arange(100, 300, 50)

tau_0 = np.arange(0.5, 5, 1)
tau_1 = np.arange(10, 100, 15)

train_gridsearch = 98 #1985Q4

parameter_set = []

for t_0 in tau_0:
    for t_1 in tau_1:
        parameter_set.append((t_0, t_1))

msfe_list = []
number_of_possibilities = len(parameter_set)
print(f'Number of possibilities: {number_of_possibilities}')

for idx, p_set in enumerate(parameter_set):
    
    tau_0, tau_1 = p_set
    
    tvp_svss = TVPVARModel(X_complete, y_complete, p, train)
    tvp_svss.initialize_priors(prior='svss',
                               prior_parameters={'tau_0': tau_0,
                                                 'tau_1': tau_1,
                                                 'pi0': 0.5},
                               )
    
    mt1t, St1t = tvp_svss.train(print_status=False)
    msfe_list.append(tvp_svss.insample_msfe())
    
    print(f'Progress: {(idx+1)}/{number_of_possibilities}')
    
# Clear printing output
clear_output()
    
sorted_indices = np.argsort(msfe_list)

for i in range(number_of_possibilities):
    
    sorted_index = sorted_indices[i]
    
    print(f'MSFE: {msfe_list[sorted_index]} | '
          f'tau_0: {parameter_set[sorted_index][0]} & '
          f'tau_1: {parameter_set[sorted_index][1]}')
    
    
optimal_svss = parameter_set[sorted_indices[0]]

MSFE: 0.00012972280170162645 | tau_0: 1.5 & tau_1: 70
MSFE: 0.00012972280170162645 | tau_0: 1.5 & tau_1: 10
MSFE: 0.00012972280170162645 | tau_0: 1.5 & tau_1: 25
MSFE: 0.00012972280170162645 | tau_0: 1.5 & tau_1: 40
MSFE: 0.00012972280170162645 | tau_0: 1.5 & tau_1: 55
MSFE: 0.00012972280170162645 | tau_0: 1.5 & tau_1: 85
MSFE: 0.00013491142271366885 | tau_0: 2.5 & tau_1: 85
MSFE: 0.00013491142271366885 | tau_0: 2.5 & tau_1: 70
MSFE: 0.00013491142271366885 | tau_0: 2.5 & tau_1: 55
MSFE: 0.00013491142271366885 | tau_0: 2.5 & tau_1: 25
MSFE: 0.00013491142271366885 | tau_0: 2.5 & tau_1: 10
MSFE: 0.00013491142271366885 | tau_0: 2.5 & tau_1: 40
MSFE: 0.00013662341271847755 | tau_0: 3.5 & tau_1: 85
MSFE: 0.00013662341271847755 | tau_0: 3.5 & tau_1: 70
MSFE: 0.00013662341271847755 | tau_0: 3.5 & tau_1: 10
MSFE: 0.00013662341271847755 | tau_0: 3.5 & tau_1: 25
MSFE: 0.00013662341271847755 | tau_0: 3.5 & tau_1: 40
MSFE: 0.00013662341271847755 | tau_0: 3.5 & tau_1: 55
MSFE: 0.00013734265922245092

In [7]:
# Horseshoe
# Optimizing the shrinkage for M = 7 (default values are optimal for M = 3)

a0_horseshoe = np.arange(10, 30, 5)
b0_horseshoe = np.arange(10, 30, 5)

train_gridsearch = 98 # 1985Q1

parameter_set = []

for a0 in a0_horseshoe:
    for b0 in b0_horseshoe:
        parameter_set.append((a0, b0))

msfe_list = []
number_of_possibilities = len(parameter_set)
print(f'Number of possibilities: {number_of_possibilities}')

for idx, p_set in enumerate(parameter_set):
    
    a0, b0 = p_set
    
    tvp_horseshoe = TVPVARModel(X_complete, y_complete, p, train_gridsearch)
    tvp_horseshoe.initialize_priors(prior='horseshoe',
                               prior_parameters={'a0': a0,
                                                 'b0': b0})
    
    mt1t, St1t = tvp_horseshoe.train(print_status=False)
    msfe_list.append(tvp_horseshoe.insample_msfe())
    
    print(f'Progress: {(idx+1)}/{number_of_possibilities}')
    
# Clear printing output
clear_output()
    
sorted_indices = np.argsort(msfe_list)

for i in range(number_of_possibilities):
    
    sorted_index = sorted_indices[i]
    
    print(f'MSFE: {msfe_list[sorted_index]} | '
          f'a0: {parameter_set[sorted_index][0]} & '
          f'b0: {parameter_set[sorted_index][1]}')
    
optimal_horseshoe = parameter_set[sorted_indices[0]]

MSFE: 0.0002989820296135748 | a0: 25 & b0: 10
MSFE: 0.0003005769103183492 | a0: 20 & b0: 10
MSFE: 0.00030065778330119405 | a0: 25 & b0: 15
MSFE: 0.00030169274512990054 | a0: 25 & b0: 20
MSFE: 0.0003017710585454447 | a0: 20 & b0: 15
MSFE: 0.0003019112611420423 | a0: 15 & b0: 10
MSFE: 0.0003023689973662344 | a0: 25 & b0: 25
MSFE: 0.00030252360808765745 | a0: 20 & b0: 20
MSFE: 0.0003027388581570997 | a0: 15 & b0: 15
MSFE: 0.0003030336237965445 | a0: 20 & b0: 25
MSFE: 0.0003030568300059647 | a0: 10 & b0: 10
MSFE: 0.00030329397623418274 | a0: 15 & b0: 20
MSFE: 0.0003036931263164263 | a0: 10 & b0: 15
MSFE: 0.00030370589928861084 | a0: 15 & b0: 25
MSFE: 0.00030420082763656136 | a0: 10 & b0: 20
MSFE: 0.00030463796351949684 | a0: 10 & b0: 25


In [11]:
# Lasso
# Optimizing the shrinkage for M = 7 (default values are optimal for M = 3)

lambda_parameters = np.arange(10, 200, 25)

train_gridsearch = 98 #1985Q4

parameter_set = []

for lambda_param in lambda_parameters:
    parameter_set.append((lambda_param))

msfe_list = []
number_of_possibilities = len(parameter_set)
print(f'Number of possibilities: {number_of_possibilities}')

for idx, p_set in enumerate(parameter_set):
    
    lambda_param = p_set
    
    tvp_lasso = TVPVARModel(X_complete, y_complete, p, train_gridsearch)
    tvp_lasso.initialize_priors(prior='lasso',
                               prior_parameters={'lambda_param': lambda_param})
    
    mt1t, St1t = tvp_lasso.train(print_status=False)
    msfe_list.append(tvp_lasso.insample_msfe())
    
    print(f'Progress: {(idx+1)}/{number_of_possibilities}')
    
# Clear printing output
clear_output()
    
sorted_indices = np.argsort(msfe_list)

for i in range(number_of_possibilities):
    
    sorted_index = sorted_indices[i]
    
    print(f'MSFE: {msfe_list[sorted_index]} | '
          f'lambda: {parameter_set[sorted_index]}')
    
optimal_lambda = parameter_set[sorted_indices[0]]

MSFE: 0.0003106824120382188 | lambda: 10
MSFE: 0.0003113822089094955 | lambda: 35
MSFE: 0.00031151153394016665 | lambda: 60
MSFE: 0.00031156576641507016 | lambda: 85
MSFE: 0.000311595476504788 | lambda: 110
MSFE: 0.00031161427128689767 | lambda: 135
MSFE: 0.00031162723584265804 | lambda: 160
MSFE: 0.0003116367186356007 | lambda: 185


In [8]:
# SVSS
# Calculate h-step forecasts (h = 1 .. 8) for M = 7

# optimal_svss = (4.5, 250) #M = 7
# optimal_svss = (1.5, 70)

tvp_svss = TVPVARModel(X_complete, y_complete, p, train)
tvp_svss.initialize_priors(prior='svss',
                           prior_parameters={'tau_0': optimal_svss[0],
                                             'tau_1': optimal_svss[1],
                                             'pi0':0.5})

h_forecast = 8

msfe_svss, alpl_svss = tvp_svss.calculate_metrics(h_forecast)

clear_output()
print(f'MSFE for SVSS:')
print(np.mean(msfe_svss,1))

MSFE for SVSS:
[  3.80619501   4.89747366   5.4702318   11.2607501   15.74260275
  17.89618376  43.67138471 117.52526827]


In [ ]:
# Horseshoe
# Calculate h-step forecasts (h = 1 .. 8) for M = 7

optimal_horseshoe = (30,30)

tvp_horseshoe = TVPVARModel(X_complete, y_complete, p, train)
tvp_horseshoe.initialize_priors(prior='horseshoe',
                            prior_parameters={'a0': optimal_horseshoe[0],
                                              'b0': optimal_horseshoe[1]})

tvp_horseshoe.print_status = False

h_forecast = 8

msfe_horseshoe, alpl_horseshoe = tvp_horseshoe.calculate_metrics(h_forecast)

clear_output()
print(f'MSFE for Horseshoe:')
print(np.mean(msfe_horseshoe,1))

In [9]:
for i in range(8):
    
    np.savetxt(f'../rcode/horseshoe_{i+1}.csv', tvp_svss.y_pred[:,:,i], delimiter=",")

In [12]:
tvp_svss = TVPVARModel(X_complete, y_complete, p, train, 200)
tvp_svss.train(print_status=False)

tvp_svss.insample_msfe()

0.009993645192407002

In [ ]:
# Lasso
# Calculate h-step forecasts (h = 1 .. 8) for M = 7

tvp_lasso = TVPVARModel(X_complete, y_complete, p, train)
tvp_lasso.initialize_priors(prior='lasso',
                            prior_parameters={'lambda_param': optimal_lambda})

h_forecast = 8

msfe_lasso, alpl_lasso = tvp_lasso.calculate_metrics(h_forecast)

clear_output()
np.mean(msfe_lasso, 1)

In [32]:
tvp = TVPVARModel(X_complete, y_complete, p, train)

In [33]:
tvp.initialize_priors(prior='horseshoe', prior_parameters={'a0':30, 'b0': 30})
tvp.train()

Iteration: 20
Elapsed time: 178.2850239276886 seconds
lambda: 0.13539244265909037,phi: 1.991685284646354, v: 1.043190251101326, delta: 1.2961856259053217 mt1t: -0.008465813538465232
Seconds for one iteration: 17.867598056793213
Difference: 1.0122176104804291e-07
lambda: 0.13539244268474962,phi: 1.9916852846696806, v: 1.0431902511011286, delta: 1.2961856258622775 mt1t: -0.008465813544679752
Seconds for one iteration: 18.073746919631958
Difference: 4.552992011233519e-08
lambda: 0.13539244267996065,phi: 1.991685284667848, v: 1.0431902511011515, delta: 1.2961856258750395 mt1t: -0.008465813543428553
Seconds for one iteration: 18.660526752471924
Difference: 1.5181563872691808e-08


(array([[-0.37507866, -0.37564453, -0.36462166, ..., -0.08688877,
         -0.08175568, -0.07957406],
        [-0.12535171, -0.11868922, -0.1172138 , ..., -0.09501075,
         -0.09259786, -0.09026482],
        [ 0.27715705,  0.26930679,  0.24299719, ...,  0.23529139,
          0.23093353,  0.22755203],
        ...,
        [ 0.07542196,  0.07522619,  0.0740648 , ...,  0.03817438,
          0.03462372,  0.03110008],
        [ 0.2227205 ,  0.22265119,  0.22243562, ...,  0.0302619 ,
          0.02990701,  0.02956047],
        [ 0.38525167,  0.38105229,  0.37548607, ...,  0.06250583,
          0.05998628,  0.0574884 ]]),
 array([[[ 0.68329173,  0.63945064,  0.6094143 , ...,  0.51155048,
           0.53137047,  0.5525445 ],
         [ 0.17270911,  0.15739551,  0.14168305, ..., -0.00786345,
          -0.01116344, -0.013964  ],
         [-0.05027901, -0.03593297, -0.03268992, ..., -0.00249042,
          -0.00153225, -0.0013647 ],
         ...,
         [-0.00020395, -0.00020075, -0.00019168

In [ ]:
import pickle

dump_to_disk = (msfe_horseshoe, alpl_horseshoe, msfe_lasso,
                alpl_lasso, optimal_svss, optimal_horseshoe, optimal_lambda)

with open('dumps/mse_alpl_7_non_standard_horseshoe_lasso.pkl', 'wb') as f:
    pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)

In [10]:
print("hello world")

hello world


In [ ]:
np.mean(msfe_horseshoe,1)

In [ ]:
import pickle

dump_to_disk = (msfe_horseshoe, alpl_horseshoe, optimal_horseshoe)

with open('dumps/mse_alpl_7_non_standard_horseshoe.pkl', 'wb') as f:
    pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)